## Get data from DT API

In [1]:
import requests
import json

api_url_base = 'https://api.staging.digitaltwin.imec-apt.be/'

#Getting the bounding box of the street grid
api_url = api_url_base + 'v1/config/map/bounds'
variable={}
headers={}
response = requests.get(api_url, headers=headers, params=variable)
# get content to json format: response.content.json()

# getting all streets

api_url = api_url_base + 'v1/roads/segments'
variable={}
headers={}
print('Fetching Wegenregister streets through Digital Twin API...')
response = requests.get(api_url, headers=headers, params=variable)
print('Done Fetching.')
streets = response.json()

Fetching Wegenregister streets through Digital Twin API...
Done Fetching.


In [2]:
streets[0]

{'id': 1,
 'originalId': 74369,
 'versionId': '74369_3',
 'geometryVersionId': '74369_3',
 'startPointId': 822796,
 'endPointId': 980069,
 'status': {'id': 4, 'value': 'in gebruik'},
 'morphology': {'id': 120, 'value': 'dienstweg'},
 'category': {'id': '-9', 'value': 'niet van toepassing'},
 'left': {'id': '-9', 'name': ''},
 'right': {'id': '-9', 'name': ''},
 'managedBy': {'id': 'AWV121',
  'value': 'Agentschap Wegen en Verkeer - District Antwerpen'},
 'organisation': {'id': 'AWV', 'value': 'Agentschap Wegen en Verkeer'},
 'accessibility': {'id': 1, 'value': 'openbare weg'},
 'geometry': {'type': 'LineString',
  'coordinates': [[151220.7320000008, 210372.2800000012],
   [151217.2940000072, 210376.6230000034],
   [151216.0820000023, 210379.4230000041],
   [151216.0090000033, 210382.6250000037],
   [151216.9210000038, 210385.7040000036],
   [151217.5030000061, 210387.6710000038],
   [151219.9470000044, 210394.8770000041],
   [151219.8770000041, 210398.2860000059],
   [151217.6330000088

Checking: outputting geojson to file to see if this is te whole grid

In [157]:
geojsonStreets = {
  "type": "FeatureCollection",
  "features": []}
for street in streets:
    geojsonStreets['features'].append({'type': 'Feature', 'properties': {}, 'geometry':{'type': 'LineString' , 'coordinates' : transformStreetToGPS(street['geometry']['coordinates'])} })

#debugging street API
with open('streets_visualisation.json', 'w') as outfile:
    json.dump(geojsonStreets, outfile)

## Convertions and handling

In [3]:
def coordinate_lambert_to_WGS84(x,y):
    """ returns an dict of lon,lat coordinates

    input: coordinaten in CRS = EPSG 3035 (Lambert)
    towards EPSG WGS 84 standaard GPS coordinaten (EPSG 4326)
    the transformation will be done using the pyProj package.
    (https://pyproj4.github.io/pyproj/dev/api/transformer.html#pyproj-transform) 
    If this fails, another try is done through an API:
    https://github.com/klokantech/epsg.io
    """
    import requests
    import pyproj
    from pyproj import transform, Proj
    
    #using pyProj package to avoid dependence on API module
    pIn = pyproj.Proj(init='epsg:31370')
    pOut = pyproj.Proj(init='epsg:4326')
    try: 
        new_x, new_y = pyproj.transform(pIn,pOut, x, y)
        return {'lon': round(new_x,10), 'lat': round(new_y,10)}
    except:
        #using API if pyProj fails
        api_token = 'your_api_token'
        api_url_base = 'http://epsg.io/trans'
        api_url = api_url_base
        headers = ''
        variable={'x': str(x), 'y': str(y), 'z' : '0',
                's_srs' : 31370, 't_srs' : 4326, 'callback':'jsonpFunction'}    
        response = requests.get(api_url, headers=headers, params=variable)

        if response.status_code == 200:
            # print(json.loads(response.content.decode('utf-8')[14:-1]))
            temp = json.loads(response.content.decode('utf-8')[14:-1])
            return {'lon': float(temp['x']), 'lat': float(temp['y'])}

        return None

In [4]:
coordinate_lambert_to_WGS84(151220, 210372)

{'lon': 4.3862131369, 'lat': 51.2033145201}

In [5]:
def transformStreetToGPS(coordinateListLambert):
    newCoordinateList = []
    for coordinate in coordinateListLambert:
        result = coordinate_lambert_to_WGS84(coordinate[0],coordinate[1])
        newCoordinateList.append([result['lon'], result['lat']])
    return newCoordinateList

In [6]:
test = transformStreetToGPS(streets[0]['geometry']['coordinates'])
print(test)

[[4.3862236112, 51.2033170353], [4.3861744359, 51.2033560797], [4.3861571045, 51.20338125], [4.3861560712, 51.2034100311], [4.3861691307, 51.2034377046], [4.3861774648, 51.2034553836], [4.3862124586, 51.203520149], [4.386211469, 51.2035507907], [4.3861793724, 51.2035779048], [4.3861745949, 51.2035812043], [4.3861413663, 51.2036041389], [4.3860986021, 51.20360997], [4.3859811426, 51.2036333938]]


In [7]:
def calculateStreetLengthGPS(coordinateList):
    from math import sin, cos, sqrt, atan2, radians
    R = 6373.0  # approximate radius of earth in km
    length = 0
    for coordinate in coordinateList[:-1]:
        nextCoordinate = coordinateList[coordinateList.index(coordinate)+1]
        
        lat1 = radians(coordinate[0])
        lon1 = radians(coordinate[1])
        lat2 = radians(nextCoordinate[0])
        lon2 = radians(nextCoordinate[1])

        dlon = lon2 - lon1
        dlat = lat2 - lat1

        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))

        distance = R * c
        length += distance
        
    #print("Result in kilometers:", length)
    return length

In [8]:
def calculateStreetLengthLambert(coordinateListLambert):
    from math import sin, cos, sqrt, atan2, radians
    length = 0
    for coordinate in coordinateListLambert[:-1]:
        nextCoordinate = coordinateListLambert[coordinateListLambert.index(coordinate)+1]
        
        lat1 = (coordinate[0])
        lon1 = (coordinate[1])
        lat2 = (nextCoordinate[0])
        lon2 = (nextCoordinate[1])
        
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        
        

        distance = sqrt(dlat**2 + dlon**2)
        #print("Pythagoras in meters:", distance)
        length += distance
        
    #print("Result in meters:", length)
    return length

In [9]:
calculateStreetLengthLambert(streets[0]['geometry']['coordinates'])

47.4515620782812

Applying following functions:
- calculate street length
- transform coordinates to GPS

In [10]:
for street in streets:
    street['SHAPE_Length'] = calculateStreetLengthLambert(street['geometry']['coordinates'])
    street['geometryGPS'] = {'type': 'LineString', 'coordinates':[]}
    street['geometryGPS']['coordinates'] = transformStreetToGPS(street['geometry']['coordinates'])
    street_length = calculateStreetLengthGPS(street['geometryGPS']['coordinates'])
    #street['SHAPE_Length'] = 1000*street_length

In [11]:
#controle
print(streets[5])

{'id': 6, 'originalId': 74410, 'versionId': '74410_1', 'geometryVersionId': '74410_1', 'startPointId': 148819, 'endPointId': 197371, 'status': {'id': 4, 'value': 'in gebruik'}, 'morphology': {'id': 114, 'value': 'wandel- of fietsweg, niet toegankelijk voor andere voertuigen'}, 'category': {'id': '-9', 'value': 'niet van toepassing'}, 'left': {'id': '-9', 'name': ''}, 'right': {'id': '-9', 'name': ''}, 'managedBy': {'id': '11002', 'value': 'Stad Antwerpen'}, 'organisation': {'id': 'AGIV', 'value': 'Agentschap voor Geografische Informatie Vlaanderen'}, 'accessibility': {'id': 1, 'value': 'openbare weg'}, 'geometry': {'type': 'LineString', 'coordinates': [[150840.7810000032, 213128.454], [150841.936999999, 213125.4070000015], [150847.75, 213117.4070000015], [150852.5939999968, 213111.938000001], [150853.9689999968, 213111.3599999994], [150854.4689999968, 213111.1409999989], [150857.4060000032, 213109.8909999989], [150859.8790000007, 213109.8669999987]]}, 'recordedAt': '2014-02-20T13:35:32

In [12]:
#debugging point
with open('data_streets.json', 'w') as outfile:
    json.dump(streets, outfile)

## Handling squares

In [13]:
##########################
# 2.2 Modality split for each street 
# can be improved using Yolo, telraam data, statistics, ...
##########################
def applyModalitySplit(js_street):
    #fucntion to calculate some modality info for street grid, taking in street grid, modifying it and pushing it back out
    from shapely.geometry import shape

    for street in js_street:
        shape_street = shape(street['geometryGPS'])
        street['properties'] = {}
        street['properties']['isSquare'] = (shape_street.geom_type != 'LineString') #marking out squares

    #street type and modality split
    for street in js_street:
    
        if (street['properties']['isSquare']):
            sigma = 0.10
            street['properties']['modal_split_car'] = 0.00
            street['properties']['modal_split_bike'] = 0.5 + sigma
            street['properties']['modal_split_pedestrian'] = 0.5 + sigma
            street['properties']['modal_split_public_transport'] = 0.00
            street['properties']['modal_split_truck'] = 0.00
            #this is a square, not a street
        
        elif (street['morphology'] == 'wandel- of fietsweg, niet toegankelijk voor andere voertuigen') :
            sigma = 0.10
            street['properties']['modal_split_car'] = 0.00
            street['properties']['modal_split_bike'] = 0.5 + sigma
            street['properties']['modal_split_pedestrian'] = 0.5 + sigma
            street['properties']['modal_split_public_transport'] = 0.00
            street['properties']['modal_split_truck'] = 0.00
            
        else:
            sigma = 0.30
            street['properties']['modal_split_car'] = 0.309 + sigma
            street['properties']['modal_split_bike'] = 0.213 + sigma
            street['properties']['modal_split_pedestrian'] = 0.083 + sigma
            street['properties']['modal_split_public_transport'] = 0.384 + sigma
            street['properties']['modal_split_truck'] = 0.011 + sigma
            # data: https://www.mobielvlaanderen.be/pdf/ovg52/samenvatting.pdf
            
            #nested if for detailed data
            if (street['category'] == 'hoofdweg') :
                street['properties']['modal_split_car'] = 0.58 + sigma
                street['properties']['modal_split_bike'] = 0.19 + sigma
                street['properties']['modal_split_pedestrian'] = 0.18 + sigma
                street['properties']['modal_split_public_transport'] = 0.01 + sigma
                street['properties']['modal_split_truck'] = 0.04 + sigma
                # data: straatvinken 2018; de Vrièrestraat [http://straatvinken.datylon.com/]
                # improvement: this counts car and bus as same number of passengers, correction should be applied

            elif (street['category'] == 'buurtweg') :
                street['properties']['modal_split_car'] = 0.58 + sigma
                street['properties']['modal_split_bike'] = 0.19 + sigma
                street['properties']['modal_split_pedestrian'] = 0.18 + sigma
                street['properties']['modal_split_public_transport'] = 0.01 + sigma
                street['properties']['modal_split_truck'] = 0.04 + sigma

            elif (street['category'] == 'wijkweg') :
                street['properties']['modal_split_car'] = 0.11 + sigma
                street['properties']['modal_split_bike'] = 0.21 + sigma
                street['properties']['modal_split_pedestrian'] = 0.68 + sigma
                street['properties']['modal_split_public_transport'] = 0.00
                street['properties']['modal_split_truck'] = 0.00
                # data: straatvinken 2018; Franckenstraat [http://straatvinken.datylon.com/]
                # improvement: this counts car and bus as same number of passengers, correction should be applied
        
        #for a simpler setup
        street['properties']['modal_split_motorized'] = street['properties']['modal_split_car'] + street['properties']['modal_split_truck'] + street['properties']['modal_split_public_transport']
        street['properties']['modal_split_non_motorized'] = street['properties']['modal_split_pedestrian'] + street['properties']['modal_split_bike']

    return js_street

In [14]:
streets = applyModalitySplit(streets)

In [56]:
##########################
### get in squares
##########################
def getSquares():
    path = '../metadata/'
    filename = 'UP_plain_squares.geojson'
    with open(path+filename) as json_file:
        squares = json.load(json_file)

    js_square = {'type': 'FeatureCollection', 'crs': {'type': 'name', 'properties': {'name': 'EPSG:4326'}}, 'features': squares['features']}

    return js_square

In [57]:
#merge squares and streets
def mergeStreetsSquares(squares, streets):
    #one final json
    #temp = to_add_squares + to_add_streets
    #js_locations = {'type': 'FeatureCollection', 'crs': {'type': 'name', 'properties': {'name': 'EPSG:4326'}}, 'features': temp}
    return None

## Outputting street grid with information to file
The file is being read by code to render the streets.

In [32]:
def streetGridToFile(js_street):
    from shapely.geometry import shape
    path = '../metadata/'
    with open(path + 'open_street_with_modalities.json', 'w') as json_file:
        json.dump(js_street, json_file)
    print(js_street[0]['properties']['modal_split_car'])
    linelist2 = list(map(lambda street: {'street_geo' : shape(street["geometryGPS"]), 'street_id' : street['originalId'],\
                                        'modal_split_car': street['properties']['modal_split_car'],\
                                        'modal_split_bike': street['properties']['modal_split_bike'],\
                                        'modal_split_pedestrian': street['properties']['modal_split_pedestrian'],\
                                        'modal_split_public_transport': street['properties']['modal_split_public_transport'],\
                                        'modal_split_truck': street['properties']['modal_split_truck'],\
                                        'modal_split_motorized': street['properties']['modal_split_motorized'],\
                                        'modal_split_non_motorized': street['properties']['modal_split_non_motorized']}, js_street))

    df_street = pd.DataFrame(columns=['street_id', 'street_geo', 'street_length', 'modal_split_car', 'modal_split_bike', 
                                    'modal_split_pedestrian', 'modal_split_public_transport', 'modal_split_truck',
                                    'modal_split_motorized', 'modal_split_non_motorized'])

    for street in linelist2:
        df_street = df_street.append({'street_id': street['street_id'], 'street_geo': street['street_geo'], 'street_length': street['SHAPE_Length'],\
                                    'modal_split_car': street['properties']['modal_split_car'],\
                                    'modal_split_bike': street['properties']['modal_split_bike'],\
                                    'modal_split_pedestrian': street['properties']['modal_split_pedestrian'],\
                                    'modal_split_public_transport': street['properties']['modal_split_public_transport'],\
                                    'modal_split_truck': street['properties']['modal_split_truck'],\
                                    'modal_split_non_motorized': street['properties']['modal_split_non_motorized'],\
                                    'modal_split_motorized': street['properties']['modal_split_motorized']}, ignore_index=True)

    df_street.to_csv(path + 'street_properties.csv')
    print('open_street_with_modalities.json written to disk, street_properties.csv written to disk.')

In [33]:
streetGridToFile(streets)

0.609


KeyError: 'modal_split_car'

## Reading in saved streetgrid file

In [ ]:
def loadStreetsFile():
    path = ''
    with open(path + 'open_street_with_modalities.json', 'r') as json_file:
        streets = json.load(json_file)
    return streets

## Reading in metadatafiles from data

In [15]:
#reference files
import pandas as pd
import shapely
from shapely.geometry import shape, Point, GeometryCollection, MultiLineString, MultiPolygon
from shapely.ops import split
import shapely.wkt
import json
import geopandas as gpd

#print(shapely.__version__)

path = 'data_januari_2020/'

df = pd.read_csv(path + 'proximus/proximus_ref.csv')

df['locationrange'] = df['locationrange'].apply(lambda x : shape(json.loads(x)).buffer(0))
gdf_proximus_ref = gpd.GeoDataFrame(df, geometry='locationrange')

df = pd.read_csv(path + 'citymesh/citymesh_ref.csv')
df['locationrange'] = df['locationrange'].apply(lambda x : shapely.wkt.loads(x))
gdf_citymesh_ref = gpd.GeoDataFrame(df, geometry='locationrange')


In [16]:
gdf_citymesh_ref.head()

,Unnamed: 0,locationid,locationrange,data_source
0,0,e4:95:6e:41:6a:ad,POLYGON Z ((4.399055863395957 51.2172988955892...,citymesh
1,1,e4:95:6e:41:6a:f7,POLYGON Z ((4.398445417648279 51.2142897748434...,citymesh
2,2,e4:95:6e:41:6a:ab,POLYGON Z ((4.398952348541532 51.2154532551113...,citymesh
3,3,e4:95:6e:41:6a:7b,POLYGON Z ((4.397615883295943 51.2124858944321...,citymesh
4,4,e4:95:6e:42:e5:9b,POLYGON Z ((4.395991231415448 51.2105765240536...,citymesh


In [17]:
frames = [gdf_proximus_ref,
          gdf_citymesh_ref,
          #gdf_ANPR_ref,
          #gdf_smart_crossing_ref
          #gdf_motion_ref,
          #gdf_camera_ref
         ]
gdf_ref = pd.concat(frames, sort=True)
gdf_ref['index'] = gdf_ref[['grid_index', 'locationid']].apply(lambda x: ','.join(x.dropna().astype(str)), axis=1)
#gdf_ref['index'] = gdf_ref[['grid_index', 'locationid']].apply(lambda x: ','.join(x.dropna().astype(str)), axis=1)

gdf_ref.drop(columns=['grid_index', 'locationid'], inplace=True)
#gdf_ref.drop(columns=['Street', 'cell_index', 'locationid', 'direction'], inplace=True)
gdf_ref.reset_index(inplace=True)
gdf_ref.drop(columns=['Unnamed: 0'], inplace=True)

gdf_ref.drop(columns=['level_0'], inplace=True)

## calculating area of shape: important for accuracy levels
gdf_ref['area'] = gdf_ref['locationrange'].apply(lambda x: x.area)

In [18]:
gdf_ref.head(900)

,data_source,locationrange,opSchelde,index,area
0,proximus,"POLYGON ((4.36259122 51.18033983, 4.36231676 5...",False,250mE392725N313400,0.000008
1,proximus,"POLYGON ((4.36231676 51.1825793, 4.36204227 51...",False,250mE392725N313425,0.000008
2,proximus,"POLYGON ((4.36615753 51.18051296, 4.36588325 5...",False,250mE392750N313400,0.000008
3,proximus,"POLYGON ((4.36972387 51.18068597, 4.36944976 5...",False,250mE392775N313400,0.000008
4,proximus,"POLYGON ((4.36588325 51.18275243, 4.36560893 5...",False,250mE392750N313425,0.000008
5,proximus,"POLYGON ((4.36944976 51.18292546, 4.36917561 5...",False,250mE392775N313425,0.000008
6,proximus,"POLYGON ((4.36204227 51.18481876, 4.36176774 5...",False,250mE392725N313450,0.000008
7,proximus,"POLYGON ((4.36176774 51.18705822, 4.36149318 5...",False,250mE392725N313475,0.000008
8,proximus,"POLYGON ((4.36149318 51.18929768, 4.36121859 5...",False,250mE392725N313500,0.000008
9,proximus,"POLYGON ((4.36533457 51.18723138, 4.36506019 5...",False,250mE392750N313475,0.000008


## actual street cutting
With streets and gdf_locations in memory, we can calculate

In [19]:
def calculateItemsLocation(gdf_ref, js_street):

    street_cuts = pd.DataFrame(columns=['street_objectid', 'street_geometry','street_length', 
                                        'street_segment_id', 'street_segment_geometry', 'street_segment_length',
                                        'data_source','data_source_index','data_source_ref_polygon', 'data_source_ref_polygon_area'])

    # cutting shapes
    polylist = list(map(lambda i: gdf_ref['locationrange'][i], range(0,len(gdf_ref))))
    polygons = MultiPolygon(polylist)

    global_area = polygons.convex_hull

    for street in js_street:
        street_shapefile = shape(street["geometryGPS"])
        splits = list(split(street_shapefile, polygons)) #split is actual shapely command to split the street according to the locationranges of data sources
        for street_segment in splits:
            i = 0
            for index, rows in gdf_ref.iterrows():
                if (rows['locationrange'].buffer(0.00001)).contains(street_segment):
                    #entry maken in tabel !!
                    street_cuts = street_cuts.append({'street_objectid': street['originalId'],
                                        'street_geometry': street_shapefile,
                                        'street_length': street['SHAPE_Length'],
                                        'street_segment_id': splits.index(street_segment),
                                        #'street_segment_source_id': i,
                                        'street_segment_geometry': street_segment,
                                        'street_segment_length': street_segment.length / street_shapefile.length * street['SHAPE_Length'],
                                        'data_source': rows['data_source'],
                                        'data_source_index': rows['index'],
                                        'data_source_ref_polygon': rows['locationrange'],
                                        'data_source_ref_polygon_area': rows['area'],
                                        'street_segment_source_id': (str(street['originalId']) + " - " 
                                                                + str(splits.index(street_segment)) + " - " +
                                                                str(rows['index']))}, ignore_index=True)

                    i += 1
    return [street_cuts, polylist]

In [20]:
def timeit_wrapper(func):
    import time
    #@wraps(func)
    def wrapper(*args, **kwargs):
        start = time.perf_counter()  # Alternatively, you can use time.process_time()
        func_return_val = func(*args, **kwargs)
        end = time.perf_counter()
        print('{0:<10}.{1:<8} : {2:<8}'.format(func.__module__, func.__name__, end - start))
        return func_return_val
    return wrapper

In [21]:
street_cuts, polylist = timeit_wrapper(calculateItemsLocation)(gdf_ref, streets)

In [ ]:
#street_objectid - street_segment_id - data_source_index
len(street_cuts['data_source_index'].unique())
len(gdf_ref['index'].unique())

### Faulty sensors (where there are no streets)
In the next piece of code we are looking for locationranges that never meet a street and as such, potentially kill the linear model.

In [ ]:
def Diff(li1, li2): 
    return (list(set(li1) - set(li2)))

def sensorsWithoutStreets(gdf_ref, js_street):
    polylist = list(map(lambda i: gdf_ref['locationrange'][i], range(0,len(gdf_ref))))
    listOfSensorsWithStreets = []
    for poly in polylist:
        hasCutStreet = False
        for street in js_street:
            street_shapefile = shape(street["geometryGPS"])
            street_shapefile.intersects(poly)
            if (street_shapefile.intersects(poly)):
                hasCutStreet = True
                #index zoeken
                index = gdf_ref['index'][polylist.index(poly)]
                listOfSensorsWithStreets.append(index)
                break
            else:
                continue

    sensorsList = list(map(lambda i: gdf_ref['index'][i], range(0,len(gdf_ref))))
    listOfSensorsWithoutStreets = Diff(sensorsList, listOfSensorsWithStreets)
    return listOfSensorsWithoutStreets

In [ ]:
listOfSensorsWithoutStreets = sensorsWithoutStreets(gdf_ref, streets)
df_faulty_sensors = pd.DataFrame(columns=['index'])
path = 'output_cleansing_script/'
df_faulty_sensors.to_csv(path + 'locations_with_no_streets.csv')

In [ ]:
path = 'output_cleansing_script/'
street_cuts.to_csv(path + 'expanded_grid_street_segments.csv')

## Calculating intersections
This method should be integrated with the cutting of the streets

In [ ]:
#calculating intersections between sets of streets/ Returns a dataframe containing all relevant information
#inputs: street network in json format
#outputs: intersection dataframe
def getIntersections_json(jsonfile):
    
    street_list = list(filter(lambda x: not(x['properties']['isSquare']),jsonfile)) #filter out not-streets
    
    intersections = pd.DataFrame(columns=['intersection_id', 'object_id', 'street_segment_source_id', 'street_segment_end'])
    intersection_counter = 0
    #intersection_list = [{'intersection_id':0, 'intersection_geo':None}]
    intersection_list = [] #empty list of known intersections to start with

    #street_list = list(filter(lambda x: not(x['properties']['isSquare']),js_street))
    for street1 in street_list:
        try: 
            print('street selected: ' + street1['left']['name']  + ' ' +  str(street1['originalId']))
        except:
            pass
        #shape_street1 = shape(street1['geometry']).buffer(0.00001)
        shape_street1 = [shape(street1['geometryGPS']).boundary[0].buffer(0.00001), shape(street1['geometryGPS']).boundary[1].buffer(0.00001)]

        for j in range(0,2):
            #each segment has 2 edges

            #check if these edges lie in an existing intersection
            knownIntersection = False
            for i in range(0,len(intersection_list)):
                if (intersection_list[i]['intersection_geo'].intersects(shape_street1[j])):
                    knownIntersection = True

            #search in all the unused intersections, following streets, if they lie on this intersection
            if (not(knownIntersection)):
                for street2 in street_list[street_list.index(street1)+1:]:
                    shape_street2 = shape(street2['geometryGPS'])
                    #if a match occurs, note the streets in the dataframe and the intersection on the list of known intersections
                    if shape_street1[j].contains(shape_street2.boundary[0]):
                        intersections = intersections.append({'intersection_id':intersection_counter, 'street_segment_end':j,
                                                              'object_id':street1['originalId']},ignore_index=True)
                        intersections = intersections.append({'intersection_id':intersection_counter, 'street_segment_end':0,
                                                              'object_id':street2['originalId']},ignore_index=True)
                        intersection_list.append({'intersection_id':intersection_counter, 'intersection_geo':shape_street1[j]})
                        try:
                            print('   Side: ' + str(j) + street2['left']['name'] + ' ' +  str(street2['originalId']))
                        except:
                            pass
                    elif shape_street1[j].contains(shape_street2.boundary[1]):
                        intersections = intersections.append({'intersection_id':intersection_counter, 'street_segment_end':j,
                                                              'object_id':street1['originalId']},ignore_index=True)
                        intersections = intersections.append({'intersection_id':intersection_counter, 'street_segment_end':1,
                                                              'object_id':street2['originalId']},ignore_index=True)
                        intersection_list.append({'intersection_id':intersection_counter, 'intersection_geo':shape_street1[j]})
                        try:
                            print('   Side: ' + str(j) + street2['left']['name'] + ' ' +  str(street2['originalId']))
                        except:
                            pass
                intersection_counter += 1

    intersections['intersection_id'] = pd.to_numeric(intersections['intersection_id'], downcast='signed')
    intersections['object_id'] = pd.to_numeric(intersections['object_id'], downcast='signed')
    intersections['street_segment_source_id'] = pd.to_numeric(intersections['street_segment_source_id'], downcast='signed')
    intersections['street_segment_end'] = pd.to_numeric(intersections['street_segment_end'], downcast='signed')
    intersections = intersections.drop_duplicates()
    return intersections

#intersections = getIntersections_json(js_street)

In [ ]:
#calculating intersections between sets of streets/ Returns a dataframe containing all relevant information
#inputs: street network in dataframe
#outputs: intersection dataframe
def getIntersections(df):
    import shapely.wkt
    from shapely.geometry import shape, Point, GeometryCollection, MultiLineString, MultiPolygon
    from shapely.ops import split
    #street_list = list(filter(lambda x: not(x['properties']['isSquare']),jsonfile['features'])) #filter out not-streets
    #correct initialisation
    df = df.reset_index(drop=True)
    
    tempdf = pd.DataFrame(columns=['geometry', 'intersection_id'])
    intersections = pd.DataFrame(columns=['intersection_id', 'object_id', 'street_segment_source_id', 'street_segment_end'])
    intersection_counter = 0
    intersection_list = [] #empty list of known intersections to start with

    #street_list = list(filter(lambda x: not(x['properties']['isSquare']),js_street['features']))
    for index1, rows1 in df.iterrows():
        if (shapely.wkt.loads(str(rows1['street_segment_geometry'])).geom_type == 'LineString'):
            #checken of het een straat is en geen plein!
            shape_street1 = [shapely.wkt.loads(str(rows1['street_segment_geometry'])).boundary[0].buffer(0.000001), 
                             shapely.wkt.loads(str(rows1['street_segment_geometry'])).boundary[1].buffer(0.000001)]
            shape_street1_no_buffer = [shapely.wkt.loads(str(rows1['street_segment_geometry'])).boundary[0], 
                                       shapely.wkt.loads(str(rows1['street_segment_geometry'])).boundary[1]]
            #print(shape_street1)
            for j in range(0,2):
                #each segment has 2 edges

                #check if these edges lie in an existing intersection
                knownIntersection = False
                for i in range(0,len(intersection_list)):
                    if (intersection_list[i]['intersection_geo'].intersects(shape_street1_no_buffer[j])):
                        knownIntersection = True
                #knownIntersection = False
                
                #search in all the unused intersections, following streets, if they lie on this intersection
                if (not(knownIntersection)):
                    df2 = df.iloc[index1+1:,:]
                    for index2, rows2 in df2.iterrows():
                        if (shapely.wkt.loads(str(rows2['street_segment_geometry'])).geom_type == 'LineString'):
                            shape_street2 = shapely.wkt.loads(str(rows2['street_segment_geometry']))
                            #if a match occurs, note the streets in the dataframe and the intersection on the list of known intersections
    
                            if shape_street1[j].contains(shape_street2.boundary[0]):
                                intersections = intersections.append({'intersection_id':intersection_counter, 'street_segment_end':j,
                                                                      'object_id':rows1['street_objectid'], 
                                                                      'street_segment_source_id':rows1['street_segment_source_id']},
                                                                     ignore_index=True)
                                intersections = intersections.append({'intersection_id':intersection_counter, 'street_segment_end':0,
                                                                      'object_id':rows2['street_objectid'], 
                                                                      'street_segment_source_id':rows2['street_segment_source_id']},
                                                                     ignore_index=True)
                                intersection_list.append({'intersection_id':intersection_counter, 'intersection_geo':shape_street1[j]})
                                tempdf = tempdf.append({'geometry': shape_street1[j].buffer(0.00005), 'intersection_id': intersection_counter},
                                                       ignore_index=True)
                                
                            elif shape_street1[j].contains(shape_street2.boundary[1]):
                                intersections = intersections.append({'intersection_id':intersection_counter, 'street_segment_end':j,
                                                                      'object_id':rows1['street_objectid'], 
                                                                      'street_segment_source_id':rows1['street_segment_source_id']},
                                                                     ignore_index=True)
                                intersections = intersections.append({'intersection_id':intersection_counter, 'street_segment_end':1,
                                                                      'object_id':rows2['street_objectid'], 
                                                                      'street_segment_source_id':rows2['street_segment_source_id']},
                                                                     ignore_index=True)
                                intersection_list.append({'intersection_id':intersection_counter, 'intersection_geo':shape_street1[j]})
                                tempdf = tempdf.append({'geometry': shape_street1[j].buffer(0.00005), 'intersection_id': intersection_counter},
                                                       ignore_index=True)

                    intersection_counter += 1

    intersections['intersection_id'] = pd.to_numeric(intersections['intersection_id'], downcast='signed')
    intersections['object_id'] = pd.to_numeric(intersections['object_id'], downcast='signed')
    #intersections['street_segment_source_id'] = pd.to_numeric(intersections['street_segment_source_id'], downcast='signed')
    intersections['street_segment_end'] = pd.to_numeric(intersections['street_segment_end'], downcast='signed')
    intersections = intersections.drop_duplicates(keep='first')
    print('Intersections calculated.')
    
    #dome debugging output - in stukken zetten (van onder 30 mb)
    tempgdf = gpd.GeoDataFrame(tempdf, geometry='geometry')
    path = 'output_cleansing_script/'
    tempgdf.to_file(path + 'debugging_intersections.geojson', driver='GeoJSON')
    
    return intersections

In [ ]:
intersections = timeit_wrapper(getIntersections)(street_cuts)
#test = str(street_cuts.at[1, 'street_segment_geometry'])
#print(test)
#shapely.wkt.loads(test)
#type(street_cuts)

In [ ]:
path = 'output_cleansing_script/'
filename = 'intersections.csv'
intersections.to_csv(path+filename)